## 5.4 池化层

- 设任意二维数组$X$的$i$行$j$列的元素为$X[x,j]$。如果我们构造的卷积核输出$Y[i,j]=1$，那么说明输入中$X[i,j]$和$X[i,j+1]$数值不一样。这可能意味着物体边缘通过这两个元素之间。
- 但实际图像里，我们感兴趣的物体不会总出现在固定的位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这会导致同一个边缘对应的输出可能出现在卷积输出$Y$中的不同位置，进而对后面的模式识别造成不便。
- 池化(pooling)层的提出是为了缓解卷积层对位置的过度敏感性。

### 5.4.1 二维最大池化层和平均池化层

- 同卷积层一样，池化层每次对输入数据的一个固定形式窗口(又称池化窗口)中的元素计算输出。
- 在二维最大池化中，池化窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。
- 二维平均池化的工作原理与二维最大池化类似，但将最大运算符替换成平均运算符。
- 池化窗口形状为$p \times q$的池化层称为$p \times q$池化层，其中的池化运算叫作$p \times q$池化。

In [2]:
from mxnet import nd
from mxnet.gluon import nn

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = nd.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i : i + p_h, j : j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i : i + p_h, j : j + p_w].mean()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2, 2))


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

In [4]:
pool2d(X, (2, 2), 'avg')


[[2. 3.]
 [5. 6.]]
<NDArray 2x2 @cpu(0)>

### 5.4.2 填充和步幅

- 同卷积层一样，池化层也可以在输入的高和宽两侧填充并调整窗口的移动步幅来改变输出形状。
- 池化层填充和步幅与卷积层填充和步幅的工作机制一样。

In [5]:
X = nd.arange(16).reshape((1, 1, 4, 4))
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
<NDArray 1x1x4x4 @cpu(0)>

- 默认情况下，MaxPool2D实例里步幅和池化窗口形状相同。

In [6]:
pool2d = nn.MaxPool2D(3)
pool2d(X)


[[[[10.]]]]
<NDArray 1x1x1x1 @cpu(0)>

- 我们可以手动指定步幅和填充。

In [7]:
pool2d = nn.MaxPool2D(3, padding=1, strides=2)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>

In [8]:
pool2d = nn.MaxPool2D((2, 3), padding=(1, 2), strides=(2, 3))
pool2d(X)


[[[[ 0.  3.]
   [ 8. 11.]
   [12. 15.]]]]
<NDArray 1x1x3x2 @cpu(0)>

### 5.4.3 多通道

- 在处理多通道输入数据时，池化层对每个输入通道分别池化，而不是像卷积层那样将各通道输入的卷积结果按通道相加。这意味着池化层的输出通道数与输入通道数相等。

In [9]:
X = nd.concat(X, X + 1, dim=1)
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]

  [[ 1.  2.  3.  4.]
   [ 5.  6.  7.  8.]
   [ 9. 10. 11. 12.]
   [13. 14. 15. 16.]]]]
<NDArray 1x2x4x4 @cpu(0)>

In [10]:
pool2d = nn.MaxPool2D(3, padding=1, strides=2)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]

  [[ 6.  8.]
   [14. 16.]]]]
<NDArray 1x2x2x2 @cpu(0)>

### 小结

- 最大池化和平均池化分别取池化窗口中输入元素的最大值和平均值作为输出。
- 池化层的一个主要作用是缓解卷积层对位置的过度敏感性。
- 可以指定池化层的填充和步幅。
- 池化层的输出通道数与输入通道数相同。